In [0]:
!pip install catboost

     |████████████████████████████████| 64.4MB 43kB/s 


In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Apr 12 03:43:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import pandas as pd
import numpy as np
from hyperopt import hp
import gc
import xgboost as xgb
import catboost as ctb
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [0]:
%%time
data = pd.read_csv('/content/drive/My Drive/Data/features.csv')
data['Date'] = pd.to_datetime(data['Date'].apply(lambda x: x.split()[0]))

CPU times: user 5.18 s, sys: 522 ms, total: 5.7 s
Wall time: 7.61 s


In [0]:
%%time
for key in data.columns:
    if 'Days' in key:
        data[key] = data[key].apply(round)

CPU times: user 433 ms, sys: 50 ms, total: 483 ms
Wall time: 484 ms


In [0]:
sel = ['Amount_Cardnum_sum_0d',
 'Cardnum_count_0d',
 'Amount_Cardnum_sum_1d',
 'Amount_Merchnum_mean_0d',
 'Amount_Merchnum_max_0d',
 'Amount_Merchnum_sum_0d',
 'Amount_Merchnum_max_1d',
 'Amount_Cardnum_Merchnum_max_0d',
 'Amount_Cardnum_Merch zip_sum_0d',
 'Amount_Cardnum_Merch zip_max_3d',
 'Amount_Cardnum_Merch zip_sum_3d',
 'Cardnum_Merch zip_count_3d',
 'Amount_Cardnum_Merch zip_sum_7d',
 'Cardnum_Merch zip_count_7d',
 'Amount_Cardnum_Merch zip_max_14d',
 'Amount_Cardnum_Merch zip_sum_14d',
 'Cardnum_Merch zip_count_14d',
 'Amount_Cardnum_Merch zip_max_30d',
 'Amount_Cardnum_Merch zip_sum_30d',
 'Amount_Cardnum_Merch state_max_1d',
 'Amount_Cardnum_Merch state_max_3d',
 'Amount_Cardnum_Merch state_sum_3d',
 'Amount_Cardnum_Merch state_max_7d',
 'count_Cardnum_0d/mean_mean_Cardnum_14d',
 'count_Cardnum_0d/mean_mean_Cardnum_30d',
 'count_Cardnum_0d/mean_count_Cardnum_30d',
 'count_Cardnum_1d/mean_mean_Cardnum_30d',
 'count_Cardnum_1d/mean_count_Cardnum_30d',
 'count_Merchnum_0d/mean_count_Merchnum_7d',
 'count_Merchnum_0d/mean_count_Merchnum_14d']

In [0]:
data = data[sel + ['Date', 'Fraud']]

In [0]:
data = data[data['Date'] > pd.to_datetime('2010-01-14')]
train = data[data['Date'] <= pd.to_datetime('2010-10-31')].copy()
test = data[data['Date'] > pd.to_datetime('2010-10-31')].copy()

In [0]:
val = train.iloc[int(len(train)*0.8):,:]
train = train.iloc[:int(len(train)*0.8),:]

In [0]:
X = train.drop(columns = ['Date', 'Fraud'])
y = train['Fraud']

In [0]:
def calculate_fdr(y_true, y_pred):
    tot = y_true.sum()
    pos = y_true[y_pred.argsort()[::-1]][:int(len(y_true) * 0.03)].sum()
    return pos / tot *100

In [0]:
def xgbrf_score(params):
    model = xgb.XGBRFClassifier(**params)
    model.fit(X, y)
    predictions = model.predict_proba(val.drop(columns = ['Date', 'Fraud']))[:,1]
    fdr = calculate_fdr(val['Fraud'].values, predictions)
    del model, predictions
    gc.collect()
    return {'loss': -1 * fdr, 'status': STATUS_OK}

In [0]:
def xgbrf_optimize(evals, trials, optimizer=tpe.suggest, random_state=0):
    space = {
        'n_estimators': hp.choice('n_estimators', np.arange(200, 600, dtype=int)),
        'eta': hp.quniform('eta', 0.025, 0.25, 0.025), # A problem with max_depth casted to float instead of int with the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
        'subsample': hp.quniform('subsample', 0.7, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.7, 1, 0.05),
        'alpha' :  hp.quniform('alpha', 0, 10, 1),
        'lambda': hp.quniform('lambda', 1, 2, 0.1),
        #'nthread': cores,
        'tree_method': 'gpu_hist',
        #'n_jobs': -1,
        'seed': random_state
    }
    best = fmin(xgbrf_score, space, algo=tpe.suggest, max_evals=evals, trials = trials)
    return best

In [14]:
%%time
trials = Trials()
n= 1000
xgbrf_best_param = xgbrf_optimize(evals = n,
                      optimizer=tpe.suggest,
                      trials = trials)

100%|██████████| 1000/1000 [1:07:33<00:00,  4.05s/it, best loss: -81.31868131868131]
CPU times: user 45min 46s, sys: 21min 48s, total: 1h 7min 35s
Wall time: 1h 7min 33s


In [0]:
xgbrf_best_param['tree_method'] = 'gpu_hist'
xgbrf_best_param['max_depth'] = max(xgbrf_best_param['max_depth'], 6)

In [16]:
xgbrf_best_param

{'alpha': 3.0,
 'colsample_bytree': 1.0,
 'eta': 0.225,
 'gamma': 0.8500000000000001,
 'lambda': 1.3,
 'max_depth': 6,
 'min_child_weight': 1.0,
 'n_estimators': 254,
 'subsample': 0.8500000000000001,
 'tree_method': 'gpu_hist'}

In [0]:
xgbrf_model = xgb.XGBRFClassifier(**xgbrf_best_param)

In [18]:
%%time
xgbrf_model.fit(X, y)

CPU times: user 1.91 s, sys: 920 ms, total: 2.83 s
Wall time: 2.83 s


XGBRFClassifier(alpha=3.0, base_score=0.5, colsample_bylevel=1,
                colsample_bynode=0.8, colsample_bytree=1.0, eta=0.225,
                gamma=0.8500000000000001, lambda=1.3, learning_rate=1,
                max_delta_step=0, max_depth=6, min_child_weight=1.0,
                missing=None, n_estimators=254, n_jobs=1, nthread=None,
                objective='binary:logistic', random_state=0, reg_alpha=0,
                reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
                subsample=0.8500000000000001, tree_method='gpu_hist',
                verbosity=1)

In [0]:
predictions = xgbrf_model.predict_proba(test.drop(columns = ['Date', 'Fraud']))[:,1]

In [23]:
xgbrf_fdr = calculate_fdr(test['Fraud'].values, predictions)
print(xgbrf_fdr)

56.98324022346368
